In [4]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
from __future__ import unicode_literals
import datetime
import requests
import feedparser
from flask import Flask, render_template, requests, make_response

app = Flask(__name__)

RSS_FEED = {"zhihu": "https://www.zhihu.com/rss",
            "netease": "http://news.163.com/special/00011K6L/rss_newsattitude.xml",
            "songshuhui": "http://songshuhui.net/feed",
            "ifeng": "http://news.ifeng.com/rss/index.xml"}

DEFAULTS = {'city': '北京',
            'publication': 'zhihu'}

WEATHERS = {"北京": 101010100,
            "上海": 101020100,
            "广州": 101280101,
            "深圳": 101280601}


def get_value_with_fallback(key):
    if request.args.get(key):
        return request.args.get(key)
    if request.cookies.get(key):
        return request.cookies.get(key)
    return DEFAULTS[key]


@app.route('/')
def home():
    publication = get_value_with_fallback('publication')
    city = get_value_with_fallback('city')

    weather = get_weather(city)
    articles = get_news(publication)

    response = make_response(render_template('home.html', articles=articles,
                                             weather=weather))

    expires = datetime.datetime.now() + datetime.timedelta(days=365)
    response.set_cookie('publication',  publication, expires=expires)
    response.set_cookie('city',  city, expires=expires)

    return response


def get_news(publication):
    feed = feedparser.parse(RSS_FEED[publication])
    return feed['entries']


def get_weather(city):
    code = WEATHERS.get(city, 101010100)
    url = "http://www.weather.com.cn/data/sk/{0}.html".format(code)

    r = requests.get(url)
    r.encoding = 'utf-8'

    data = r.json()['weatherinfo']
    return dict(city=data['city'], temperature=data['temp'],
                description=data['WD'])


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

ModuleNotFoundError: No module named 'requests'

In [5]:
with open("./josn.txt", 'rb') as f:
    for line in f.readlines():
        print(line.strip())

b'\x80\x03}q\x00(X \x00\x00\x0034dd3c04020e41da73c0d3835de06697q\x01X\x15\x00\x00\x00 /home/todd/josn.txt'
b'q\x02X \x00\x00\x00fb66f772a3adf436239e8f774b51c070q\x03X\x1c\x00\x00\x00 /home/todd/.python-version'
b'q\x04X \x00\x00\x00754eee6573a9f6757b108b5127e636e8q\x05X\x1a\x00\x00\x00 /home/todd/eshop-eer.png'
b'q\x06X \x00\x00\x00606db45736321e55337a68caa7c2bc3fq\x07X\x17\x00\x00\x00 /home/todd/id_rsa.pub'
b'q\x08X \x00\x00\x0096652cb14aa3e6b7e13aff815262f36dq\tX\x13\x00\x00\x00 /home/todd/id_rsa'
b'q'
b'X \x00\x00\x00a5890ace30a3e84d9118196c161aeec2q\x0bX\x19\x00\x00\x00 /home/todd/synctest.txt'
b'q\x0cX \x00\x00\x00fa6c81e3b299047f329a92e22ad50b80q\rX \x00\x00\x00 /home/todd/week 5 excise.ipynb'
b'q\x0eX \x00\x00\x001327bac3ce1ff6fd292a93ed3369cb77q\x0fX\x18\x00\x00\x00 /home/todd/hello.ipynb'
b'q\x10X \x00\x00\x002c3098354ae1ae841b00c5ebc7f89233q\x11X\x18\x00\x00\x00 /home/todd/credentials'
b'q\x12X \x00\x00\x00ef5f3e0f5b6a0117367760a517c036dfq\x13X\x15\x00\x00\x00 /home/todd/hell

In [3]:
from io import StringIO
f = StringIO('Hello!\nHi!\nGoodbye!')
while True:
     s = f.read()
     if s == '':
         break
     print(s)

Hello!
Hi!
Goodbye!


In [7]:
import os

In [8]:
os.uname()

posix.uname_result(sysname='Linux', nodename='ip-172-31-23-17', release='4.14.77-70.59.amzn1.x86_64', version='#1 SMP Mon Nov 12 22:02:45 UTC 2018', machine='x86_64')

In [18]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    p = Pool(2)
    print(p.map(f, [1, 2, 3]))

[1, 4, 9]


Process ForkPoolWorker-50:
Process ForkPoolWorker-51:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/.pyenv/versions/3.6.8/lib/python3.6/multiprocessing

In [12]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 13565.
Run task 2 (14058)...
Run task 1 (14057)...
Waiting for all subprocesses done...
Run task 0 (14056)...
Run task 3 (14059)...
Task 0 runs 0.41 seconds.
Run task 4 (14056)...
Task 1 runs 1.02 seconds.
Task 3 runs 1.48 seconds.
Task 2 runs 1.52 seconds.
Task 4 runs 1.81 seconds.
All subprocesses done.
